
1. Compute **bigram** counts in the corpora, ignoring bigrams which contain at least one token that is not a word
   (it contains characters other than letters). The text has to be properly normalized before the counts are computed:
   it should be downcased and all punctuation should be removed. Given the sentence: "The quick brown fox jumps over the
   lazy dog", the bigram counts are as follows:
   1. "the quick": 1
   1. "quick brown": 1
   1. "brown fox": 1
   1. etc.
1. Use [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) to compute the measure 
   for all pairs of words. 
1. Sort the word pairs according to that measure in the descending order and display 30 top results.
1. Use [log likelihood ratio](http://tdunning.blogspot.com/2008/03/surprise-and-coincidence.html) (LLR) to compute the measure
   for all pairs of words.
1. Sort the word pairs according to that measure in the descending order and display 30 top results.

In [88]:
import os
import requests
import regex
from collections import Counter
import math

def filesNames():
    path = '../ustawy'
    absolute_path = os.path.realpath(path) + "\\"
    return [(absolute_path + filename, filename) for filename in os.listdir(path)]

def getFileTextRaw(path):
    with open(path, 'r', encoding="utf8") as content_file:
        return content_file.read()
    
def cleanText(text):
    lower_text = text.replace('-\n', '').replace('\n', ' ').lower()
    unpunct_text = regex.sub(r"\p{P}", ' ' ,lower_text)
    without_empty_text = regex.sub(r"\s+", ' ' ,unpunct_text).strip()
    return without_empty_text

def splitWords(text):
    return text.split(' ')

def correctWord(word):
    return word.isalpha()

def countsBigrams(words, bigram_counter, left_counter, right_counter):
    bigram_size = 0
    for index, right_word in enumerate(words[1:]):
        left_word = words[index-1]
        if correctWord(right_word) and correctWord(left_word):
            bigram_counter[(left_word, right_word)] += 1
            left_counter[left_word] +=1
            right_counter[right_word] +=1
            bigram_size += 1
    return bigram_size

def shannon(values, N):
    return sum([k / N * math.log(k/N + (k == 0)) for k in values])

def pmi(bigram, val, left_counter, right_counter, bigram_size):
    return math.log((val*bigram_size)/(left_counter[bigram[0]]*right_counter[bigram[1]]))

def llr(bigram, val, left_counter, right_counter, bigram_size):
    k_11 = val
    k_12 = right_counter[bigram[1]] - val
    k_21 = left_counter[bigram[0]] - val
    k_22 = bigram_size - right_counter[bigram[1]] - left_counter[bigram[0]] + val
    return 2 * bigram_size * (shannon([k_11, k_12, k_21, k_22], bigram_size) - shannon([k_11 + k_12, k_21 + k_22], bigram_size) - shannon([k_11 + k_21, k_12 + k_22], bigram_size))


In [89]:
bigram_counter = Counter()
left_counter = Counter()
right_counter = Counter()
bigram_size = 0
for (path, filename) in filesNames():
    content = getFileTextRaw(path)
    clean_text = cleanText(content)
    words = splitWords(clean_text)
    bigram_size += countsBigrams(words, bigram_counter, left_counter, right_counter)

In [95]:
pmi_values = []
llr_values = []
for bigram, val in bigram_counter.items():
    pmi_values.append((bigram, pmi(bigram, val, left_counter, right_counter, bigram_size)))
    llr_values.append((bigram, llr(bigram, val, left_counter, right_counter, bigram_size)))
    
pmi_values.sort(key=lambda x: -x[1])
llr_values.sort(key=lambda x: -x[1])

In [96]:
pmi_values[:30]

[(('â', 'ť'), 14.991845326195742),
 (('č', 'á'), 14.991845326195742),
 (('jednoosiowe', 'dwuosiowe'), 14.991845326195742),
 (('mierzenia', 'tętniczego'), 14.991845326195742),
 (('mundurowe', 'zuchów'), 14.991845326195742),
 (('zuchów', 'harcerzy'), 14.991845326195742),
 (('łączniki', 'instala'), 14.991845326195742),
 (('zbiornikowe', 'wyłą'), 14.991845326195742),
 (('rozdzielczych', 'sterowni'), 14.991845326195742),
 (('aluminiowe', 'stalowo'), 14.991845326195742),
 (('polistyrenu', 'styro'), 14.991845326195742),
 (('otworowa', 'okręto'), 14.991845326195742),
 (('chirurgicznymi', 'terapeutycznymi'), 14.991845326195742),
 (('wyczerpująco', 'znawca'), 14.991845326195742),
 (('przykładem', 'przykładami'), 14.991845326195742),
 (('ornament', 'kolorystyczna'), 14.991845326195742),
 (('niewielkim', 'używała'), 14.991845326195742),
 (('jednoroczne', 'kilkuletnie'), 14.991845326195742),
 (('profesorem', 'doktorem'), 14.991845326195742),
 (('napisami', 'rysunkami'), 14.991845326195742),
 (('pos

In [97]:
llr_values[:30]

[(('nr', 'poz'), 442015.687847785),
 (('o', 'mowa'), 236957.99311387417),
 (('z', 'r'), 110797.77464841149),
 (('poz', 'nr'), 94571.8434153988),
 (('art', 'ust'), 84422.68761048165),
 (('mowa', 'ust'), 81666.18210358651),
 (('mowa', 'art'), 62571.5838482982),
 (('których', 'w'), 61691.24411003185),
 (('właściwy', 'spraw'), 49784.834826818565),
 (('ust', 'pkt'), 48342.47770833059),
 (('którym', 'w'), 40834.94693266125),
 (('zastępuje', 'wyrazami'), 39037.67375870713),
 (('poz', 'z'), 38776.822326526206),
 (('określi', 'drodze'), 38083.335172063504),
 (('stosuje', 'odpowiednio'), 35081.66116709625),
 (('ustawie', 'dnia'), 32412.576681543367),
 (('wejścia', 'życie'), 31911.969356743804),
 (('poz', 'i'), 30710.237571577756),
 (('minister', 'do'), 27699.724904586416),
 (('w', 'rozporządzenia'), 27277.495598228248),
 (('wprowadza', 'następujące'), 27216.360156699935),
 (('dz', 'nr'), 25080.52686877999),
 (('której', 'w'), 23286.24848648526),
 (('dz', 'z'), 22755.626099292087),
 (('ust', 'otr

1. Answer the following questions:
   1. Which measure works better for the problem?
   1. What would be needed, besides good measure, to build a dictionary of multiword expressions?
   1. Can you identify a certain threshold which clearly divides the *good* expressions from the *bad*?

A: 
PMI was not working, maybe the solution for it would be to make a threshold how many times should the bigram appear to use it, because currently the PMI prefer bigrams with word that only appear once(strange words). LLR is working better thatn PMI but not great either.

B: As always the more data we have the better for us. Good text without spellings erros is very important. We could also use lematization to merge expressions, changed by inflection.

C: I think that the top 30 results have both bat and good expressions, so th